In [32]:
#pip install nltk

  Using cached nltk-3.6.5-py3-none-any.whl (1.5 MB)
  Using cached click-8.0.3-py3-none-any.whl (97 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
     |████████████████████████████████| 288 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import re, string
import time
import json
from nltk import tokenize
import math
import customStopWordList
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


Setting up wiki

In [65]:
with open('Computed/wiki_dict.json') as json_file:
    wiki_dict = json.load(json_file)
# wiki = pd.read_csv('Computed/wiki.csv')[['content','title','id']]
wiki = pd.read_csv('project_1_Wiki_sample.csv')[['content','title','id']]

print(wiki)

                                                   content  \
0        Morocco–Saudi Arabia relations\r\n\r\nMoroccan...   
1        Anthony United States Post Office\r\n\r\nThe A...   
2        Dumraon (Vidhan Sabha constituency)\r\n\r\nDum...   
3        Chaker Khazaal\r\n\r\nChaker Khazaal (born 28 ...   
4        Vicente Pascual Pastor\r\n\r\nVicente Pascual ...   
...                                                    ...   
1286259  Toppr\r\n\r\nToppr is a product of Haygot Educ...   
1286260  Outline of education\r\n\r\nThe following outl...   
1286261  Raell Padamsee\r\n\r\nRaell Padamsee is an Ind...   
1286262  Megan Callaway\r\n\r\nMegan Callaway is a foun...   
1286263  Diane Reay\r\n\r\nDiane Reay is a sociologist ...   

                                       title       id  
0             Morocco–Saudi Arabia relations        1  
1          Anthony United States Post Office        2  
2        Dumraon  Vidhan Sabha constituency         3  
3                             C

In [66]:
def lower_case_str(string_to_be_lower_cased):
    lowercase_str_result = str(string_to_be_lower_cased).lower()
    return lowercase_str_result

def remove_punc(string_has_punc):
    no_punc_result = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', str(string_has_punc))
    return no_punc_result

def filter_tokens(str_to_be_filtered):
    stopword_list = customStopWordList.get_custom_sw_list()
    filtered_tokens = ""
    for w in str(str_to_be_filtered).split(" "):
        if w not in stopword_list:
            filtered_tokens += w + " "
    return filtered_tokens
porter = PorterStemmer()
def stem_tokens(str_to_be_stemmed):
    stemmed_str = ""
    for w in str(str_to_be_stemmed).split(" "):
        stemmed_str += porter.stem(w) + " "
    return stemmed_str

Splitting Documents into sentences

In [67]:
def splitDoc(s):
    sentence_spliter = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', str (s))
    return sentence_spliter

df1 = wiki
# df1['content'] = wiki['content'].apply(splitDoc)
df1

,content,title,id
0,Morocco–Saudi Arabia relations\r\n\r\nMoroccan...,Morocco–Saudi Arabia relations,1
1,Anthony United States Post Office\r\n\r\nThe A...,Anthony United States Post Office,2
2,Dumraon (Vidhan Sabha constituency)\r\n\r\nDum...,Dumraon Vidhan Sabha constituency,3
3,Chaker Khazaal\r\n\r\nChaker Khazaal (born 28 ...,Chaker Khazaal,4
4,Vicente Pascual Pastor\r\n\r\nVicente Pascual ...,Vicente Pascual Pastor,5
...,...,...,...
1286259,Toppr\r\n\r\nToppr is a product of Haygot Educ...,Toppr,1286260
1286260,Outline of education\r\n\r\nThe following outl...,Outline of education,1286261
1286261,Raell Padamsee\r\n\r\nRaell Padamsee is an Ind...,Raell Padamsee,1286262
1286262,Megan Callaway\r\n\r\nMegan Callaway is a foun...,Megan Callaway,1286263


In [68]:
df1['content'].iloc[0]

"Morocco–Saudi Arabia relations\r\n\r\nMoroccan–Saudi Arabian relations refers to the current and historical relations between Morocco and Saudi Arabia. Morocco has an embassy in Riyadh and Saudi Arabia has an embassy in Rabat.\r\n\r\nBoth two nations have a long and traditional friendly relations based on many historical commons. Both Morocco and Saudi Arabia are two monarch states ruled by respective families, the Alaouites and the Al-Sauds, adhere to Sunni Islam, fear of growing Iranian and Shiite influence and their relationship is mostly described as excellent.\r\n\r\nMorocco and Saudi Arabia have together taken steps to curb Iranian influence in the Arab world, although Morocco has a moderate approach to Iran while Saudi Arabia is more cautious and hostile of Iran.\r\n\r\nSaudi Arabia has recently supported Morocco over the dispute of Western Sahara and doesn't recognize the legitimacy of Sahrawi Arab Democratic Republic, through the Arab League. Saudi Arabia, on the other side, 

In [69]:
sample = 'morocco saudi arabia relations moroccan. saudi arabian relat S. refer current histor. relat morocco saudi arabia morocco embassi riyadh saudi arabia embassi rabat both nation long tradit friendli relat base histor common morocco saudi arabia monarch rule respect famili alaouit saud adher sunni islam fear grow iranian shiit influenc relationship excel morocco saudi arabia. step curb 3.14 iranian influenc arab morocco moder approach iran saudi arabia cautiou hostil W. iran saudi arabia support morocco disput western sahara recogn legitimaci sahrawi arab democrat republ arab leagu saudi arabia help fund project region support moroccan author dure current yemeni civil war morocco particip anti houthi coalit led saudi arabia 6 plane 1 500 troop morocco neutral stanc conflict hope resolut benefit qatar saudi. arabia particip morocco anti houthi campaign morocco offer mediat slow tension gulf arab.'
tokenize.sent_tokenize(sample)

['morocco saudi arabia relations moroccan.',
 'saudi arabian relat S. refer current histor.',
 'relat morocco saudi arabia morocco embassi riyadh saudi arabia embassi rabat both nation long tradit friendli relat base histor common morocco saudi arabia monarch rule respect famili alaouit saud adher sunni islam fear grow iranian shiit influenc relationship excel morocco saudi arabia.',
 'step curb 3.14 iranian influenc arab morocco moder approach iran saudi arabia cautiou hostil W. iran saudi arabia support morocco disput western sahara recogn legitimaci sahrawi arab democrat republ arab leagu saudi arabia help fund project region support moroccan author dure current yemeni civil war morocco particip anti houthi coalit led saudi arabia 6 plane 1 500 troop morocco neutral stanc conflict hope resolut benefit qatar saudi.',
 'arabia particip morocco anti houthi campaign morocco offer mediat slow tension gulf arab.']

## NEW TFIDF FOR SENTENCES

### Query TF 

#### Takes: a word and a sentence

#### Returns: a dataframe of the document split up in sentences

In [93]:
def sentenceTF(word, sentences):
    content = []
    wordCounts = []
    sentenceCount = []
    wordCount = 0
    sentence = tokenize.sent_tokenize(sentences)
    for s in sentence:
        wordCount = 0
        if word in s.split(" "):
            wordCount += 1
        content.append(s)
        wordCounts.append(wordCount)
        sentenceCount.append(len(s.split(" ")))
    df2 = pd.DataFrame(content, columns =['content'])
    df2['WordCounts'] = wordCounts
    df2['NumWords'] = sentenceCount
    df2['TF'] = (df2['WordCounts']/df2['NumWords'])
    return df2

word = 'morocco'
sample = 'morocco saudi arabia relations moroccan. saudi arabian relat S. refer current histor. relat morocco saudi arabia morocco embassi riyadh saudi arabia embassi rabat both nation long tradit friendli relat base histor common morocco saudi arabia monarch rule respect famili alaouit saud adher sunni islam fear grow iranian shiit influenc relationship excel morocco saudi arabia. step curb 3.14 iranian influenc arab morocco moder approach iran saudi arabia cautiou hostil W. iran saudi arabia support morocco disput western sahara recogn legitimaci sahrawi arab democrat republ arab leagu saudi arabia help fund project region support moroccan author dure current yemeni civil war morocco particip anti houthi coalit led saudi arabia 6 plane 1 500 troop morocco neutral stanc conflict hope resolut benefit qatar saudi. arabia particip morocco anti houthi campaign morocco offer mediat slow tension gulf arab.'

df3 = sentenceTF(word, sample)
print(df3)

                                             content  WordCounts  NumWords  \
0           morocco saudi arabia relations moroccan.           1         5   
1       saudi arabian relat S. refer current histor.           0         7   
2  relat morocco saudi arabia morocco embassi riy...           1        42   
3  step curb 3.14 iranian influenc arab morocco m...           1        67   
4  arabia particip morocco anti houthi campaign m...           1        13   

         TF  
0  0.200000  
1  0.000000  
2  0.023810  
3  0.014925  
4  0.076923  


### Query IDF 

#### Takes: an array of wordCount from Query TF above 
#### Returns: a single float variable

In [94]:
def sentenceIDF(WordCounts):
    numSentences = len(df2)
    nw = 0
    for WordCount in WordCounts:
        if int(WordCount) > 0:
            nw += 1
            
    return math.log2(numSentences/nw) # idf

print(sentenceIDF(df2['WordCounts']))

0.32192809488736235


### Query TFIDF(returns a list of individual sentences TFIDF)

#### Takes: a word and a sentence

#### Returns: a list of TFIDF for that word and the sentence

In [95]:
def getSentenceTFIDF(word, sentence):
    return sentenceTF(word, sentence)['TF'] * (idf)

print (getSentenceTFIDF('morocco', sample))

0    0.064386
1    0.000000
2    0.007665
3    0.004805
4    0.024764
Name: TF, dtype: float64


In [96]:
AllWordsTest['content'].iloc[0]
strr = "Morocco–Saudi Arabia relations\r\n\r\nMoroccan–Saudi Arabian relations refers to the current and historical relations between Morocco and Saudi Arabia. Morocco has an embassy in Riyadh and Saudi Arabia has an embassy in Rabat.\r\n\r\nBoth two nations have a long and traditional friendly relations based on many historical commons. Both Morocco and Saudi Arabia are two monarch states ruled by respective families, the Alaouites and the Al-Sauds, adhere to Sunni Islam, fear of growing Iranian and Shiite influence and their relationship is mostly described as excellent.\r\n\r\nMorocco and Saudi Arabia have together taken steps to curb Iranian influence in the Arab world, although Morocco has a moderate approach to Iran while Saudi Arabia is more cautious and hostile of Iran.\r\n\r\nSaudi Arabia has recently supported Morocco over the dispute of Western Sahara and doesn't recognize the legitimacy of Sahrawi Arab Democratic Republic, through the Arab League. Saudi Arabia, on the other side, also helps funding several projects in the region with support from Moroccan authorities.\r\n\r\nDuring the current Yemeni Civil War, Morocco participates as part of the anti-Houthi coalition led by Saudi Arabia, with 6 planes and 1,500 troops.\r\n\r\nMorocco has taken neutral stance in the conflict, hoping for a better resolution that benefit Qatar and Saudi Arabia, the former had participated together with Morocco in anti-Houthi campaign. Morocco even offers to act as a mediator to slow the tensions between two Gulf Arab states."
print(tokenize.sent_tokenize(strr))

['Morocco–Saudi Arabia relations\r\n\r\nMoroccan–Saudi Arabian relations refers to the current and historical relations between Morocco and Saudi Arabia.', 'Morocco has an embassy in Riyadh and Saudi Arabia has an embassy in Rabat.', 'Both two nations have a long and traditional friendly relations based on many historical commons.', 'Both Morocco and Saudi Arabia are two monarch states ruled by respective families, the Alaouites and the Al-Sauds, adhere to Sunni Islam, fear of growing Iranian and Shiite influence and their relationship is mostly described as excellent.', 'Morocco and Saudi Arabia have together taken steps to curb Iranian influence in the Arab world, although Morocco has a moderate approach to Iran while Saudi Arabia is more cautious and hostile of Iran.', "Saudi Arabia has recently supported Morocco over the dispute of Western Sahara and doesn't recognize the legitimacy of Sahrawi Arab Democratic Republic, through the Arab League.", 'Saudi Arabia, on the other side, al

### For every term TFIDF (Term/resource)

Calculates a document's sentences into a word/resource TFIDF table

1. Get all the words within the document.
2. Calculate the TFIDF for those words in each sentence. (Missing words will have 0s)

#### Takes: A dataframe, and a doc_id
#### Returns: Another dataframe of TFIDFs and the sentences

In [98]:
def getAllWords(dataframe, doc_id):
    dict = {}
    for word in dataframe['content'].iloc[doc_id].split(" "):
        dict[word] = 0
#     print(list(dict.keys()))
    return list(dict.keys())

def TFIDFAllWords(dataframe, words, doc_id): 
#     print("Here are list of words")
#     print(len(words))
    TFDocuments = pd.DataFrame()
    
    count = 0
    first = True
    for word in words:
        if first:
            sentenceID = []
            print("Mark")
#             print(dataframe['content'].iloc[count])
#             print(getSentenceTFIDF(word, dataframe['content'].iloc[count]))
            for i in range(len(getSentenceTFIDF(word, dataframe['content'].iloc[count]))):
                sentenceID.append(i)
            TFDocuments = pd.DataFrame(sentenceID, columns=['Sentence_ID'])
            first = False
        TFDocuments[word] = getSentenceTFIDF(word, dataframe['content'].iloc[count])
    return TFDocuments

def secondHalf(dataframe, doc_id):
    words = getAllWords(dataframe, doc_id)
    NeedsSum = TFIDFAllWords(dataframe, words, doc_id)
    
    retVals = []
    for sentenceID in range(len(NeedsSum)):
        retVal = 0
        for word in words:
            retVal += float(NeedsSum[word].iloc[sentenceID])**2
        retVals.append(retVal)
    return retVals

AllWordsTest = wiki.iloc[0:10]
print(secondHalf(AllWordsTest, 0))


Mark
[0.005379119287775426, 0.004758873900511527, 0.006909179885187102, 0.0024225028085389415, 0.0025102556676285313, 0.0033728245001653015, 0.0048804456252706675, 0.0037784577497116992, 0.003204019209539797, 0.005737727240293787]


/var/folders/kb/7q1qk5ds74sdxqbgl26pm2vw0000gp/T/ipykernel_81019/471501796.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  TFDocuments[word] = getSentenceTFIDF(word, dataframe['content'].iloc[count])


DENOMINATOR

In [16]:
def firstHalf(sentence):
    firsthalf = math.sqrt(len(sentence)*1)
    return (firsthalf)



query = "morroco saudi"
secondHalf(query ,0)

TypeError: getSentenceTFIDF() missing 1 required positional argument: 'sentence'

COSINE SIMULARITY

In [ ]:
def cosineSimularity(word, docID):
    for doc in df:
        splitDoc();
        for sectence in doc:
            sentence 